# 🏥 Image Segmentation: U-Net, Mask R-CNN & Detectron2

Medical imaging and autonomous driving segmentation.

## Learning Outcomes
- Semantic vs instance segmentation
- U-Net architecture for medical images
- Mask R-CNN for instance segmentation
- Detectron2 for production deployment

**Level**: Advanced | **Time**: 90 min | **GPU**: Recommended

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 1. U-Net Architecture (Medical Imaging)

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    def __init__(self, n_channels=3, n_classes=1):
        super().__init__()
        # Encoder
        self.enc1 = DoubleConv(n_channels, 64)
        self.enc2 = DoubleConv(64, 128)
        self.enc3 = DoubleConv(128, 256)
        self.enc4 = DoubleConv(256, 512)
        self.pool = nn.MaxPool2d(2)
        
        # Bottleneck
        self.bottleneck = DoubleConv(512, 1024)
        
        # Decoder
        self.up4 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.dec4 = DoubleConv(1024, 512)
        self.up3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.dec3 = DoubleConv(512, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec2 = DoubleConv(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1 = DoubleConv(128, 64)
        
        self.out = nn.Conv2d(64, n_classes, 1)
    
    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        e4 = self.enc4(self.pool(e3))
        
        # Bottleneck
        b = self.bottleneck(self.pool(e4))
        
        # Decoder with skip connections
        d4 = self.dec4(torch.cat([self.up4(b), e4], dim=1))
        d3 = self.dec3(torch.cat([self.up3(d4), e3], dim=1))
        d2 = self.dec2(torch.cat([self.up2(d3), e2], dim=1))
        d1 = self.dec1(torch.cat([self.up1(d2), e1], dim=1))
        
        return torch.sigmoid(self.out(d1))

model = UNet(n_channels=3, n_classes=1).to(device)
print(f"U-Net Parameters: {sum(p.numel() for p in model.parameters()):,}")

## 2. Training U-Net

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, smooth=1.):
        super().__init__()
        self.smooth = smooth
    
    def forward(self, pred, target):
        pred = pred.view(-1)
        target = target.view(-1)
        intersection = (pred * target).sum()
        return 1 - (2. * intersection + self.smooth) / (pred.sum() + target.sum() + self.smooth)

def dice_coefficient(pred, target):
    pred = (pred > 0.5).float()
    intersection = (pred * target).sum()
    return (2. * intersection) / (pred.sum() + target.sum() + 1e-8)

# Create synthetic data
def create_synthetic_data(batch_size=4, size=256):
    images = torch.randn(batch_size, 3, size, size)
    masks = torch.zeros(batch_size, 1, size, size)
    for i in range(batch_size):
        cx, cy = np.random.randint(50, size-50, 2)
        r = np.random.randint(20, 50)
        y, x = np.ogrid[:size, :size]
        mask = ((x - cx)**2 + (y - cy)**2 <= r**2).astype(np.float32)
        masks[i, 0] = torch.from_numpy(mask)
    return images.to(device), masks.to(device)

# Training loop
criterion = DiceLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

print("Training U-Net...")
for epoch in range(5):
    model.train()
    images, masks = create_synthetic_data(8)
    optimizer.zero_grad()
    pred = model(images)
    loss = criterion(pred, masks)
    loss.backward()
    optimizer.step()
    dice = dice_coefficient(pred, masks)
    print(f"Epoch {epoch+1}: Loss={loss.item():.4f}, Dice={dice.item():.4f}")

## 3. Mask R-CNN with TorchVision

In [ ]:
from torchvision.models.detection import maskrcnn_resnet50_fpn, MaskRCNN_ResNet50_FPN_Weights

# Load pretrained Mask R-CNN
weights = MaskRCNN_ResNet50_FPN_Weights.DEFAULT
mask_rcnn = maskrcnn_resnet50_fpn(weights=weights).to(device)
mask_rcnn.eval()

categories = weights.meta['categories']
print(f"Mask R-CNN can segment: {len(categories)} classes")
print(f"Examples: {categories[:5]}")

In [ ]:
def segment_image(model, image_tensor, threshold=0.5):
    """Run instance segmentation."""
    with torch.no_grad():
        predictions = model([image_tensor.to(device)])[0]
    
    mask = predictions['scores'] > threshold
    return {
        'boxes': predictions['boxes'][mask],
        'labels': predictions['labels'][mask],
        'masks': predictions['masks'][mask],
        'scores': predictions['scores'][mask]
    }

# Demo with random image
test_img = torch.rand(3, 480, 640)
result = segment_image(mask_rcnn, test_img)
print(f"Detected {len(result['boxes'])} objects")

## 4. Detectron2 (Facebook AI)

In [ ]:
try:
    from detectron2 import model_zoo
    from detectron2.engine import DefaultPredictor
    from detectron2.config import get_cfg
    
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    
    predictor = DefaultPredictor(cfg)
    print("✅ Detectron2 loaded successfully")
except ImportError:
    print("Install Detectron2: pip install detectron2")

## 5. Model Comparison

In [ ]:
import pandas as pd

comparison = pd.DataFrame({
    'Model': ['U-Net', 'U-Net++', 'Mask R-CNN', 'DeepLab v3+', 'SegFormer'],
    'Type': ['Semantic', 'Semantic', 'Instance', 'Semantic', 'Semantic'],
    'mIoU (COCO)': ['-', '-', '37.1', '82.1', '84.0'],
    'Speed (FPS)': ['50', '40', '5', '10', '15'],
    'Best For': ['Medical', 'Medical++', 'Detection+Seg', 'General', 'Transformer']
})

print("📊 Segmentation Model Comparison:")
display(comparison)

## 6. Production Deployment

In [ ]:
# Export U-Net to ONNX
dummy_input = torch.randn(1, 3, 256, 256).to(device)
torch.onnx.export(model, dummy_input, 'unet_segmentation.onnx',
                  input_names=['image'], output_names=['mask'])
print("✅ Model exported to ONNX")

# Cost analysis
print("\n💰 Cloud Deployment Costs:")
print("  AWS p3.2xlarge (V100): ~$3.06/hour")
print("  GCP n1-standard-8 + T4: ~$0.95/hour")
print("  Azure NC6 (K80): ~$0.90/hour")

## 🎯 Key Takeaways
1. U-Net for medical imaging (skip connections key)
2. Mask R-CNN for instance segmentation
3. Dice loss for class imbalance
4. Detectron2 for production

## 📚 Further Reading
- Ronneberger et al., "U-Net" (2015)
- He et al., "Mask R-CNN" (2017)
- Chen et al., "DeepLab" series